In [22]:
#load weather and flight datasets
import pandas as pd

In [23]:
def sanity_check(df):
    month = df['MONTH'].unique()
    day = df['DAY'].unique()
    day_of_week = df['DAY_OF_WEEK'].unique()
    day_of_year = df['day_of_year'].unique()
    print(f'{len(month)} months')
    print(f'{len(day)} days')
    print(f'{len(day_of_week)} days of week')
    print(f'{len(day_of_year)} days of year')

    #make sure there are 12 months, 31 days, 24 hours
    assert len(month) == 12
    assert len(day) == 31
    assert len(day_of_week) == 7
    assert len(day_of_year) == 365


In [24]:
def dict_freq(df, col:str):
    val = {}
    for d in df[col].unique():
        val[d] = len(df[df[col] == d])
    return val

In [25]:
df_w = pd.read_csv('../datasets/weather_10.csv')

In [26]:
#change column name 'filename' to 'airport', remove the .csv extension
df_w['airport'] = df_w['filename'].str.replace('.csv', '')
#drop the 'filename' column
df_w.drop('filename', axis=1, inplace=True)

C:\Users\tccox\AppData\Local\Temp\ipykernel_8020\821801327.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_w['airport'] = df_w['filename'].str.replace('.csv', '')


In [33]:
df_f = pd.read_csv('../datasets/flights_7.csv')

C:\Users\tccox\AppData\Local\Temp\ipykernel_8020\4022389923.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_f = pd.read_csv('../datasets/flights_7.csv')


In [34]:
#print unique values of origin airport
print(len(df_f['ORIGIN_AIRPORT'].unique()))

931


In [7]:
#columns
df_f.columns
#print unique values for DAY column
df_f['MONTH'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [28]:
#remove the 'DATE' column
if 'DATE' in df_f.columns:
    df_f.drop('DATE', axis=1, inplace=True)


#add 'DAY_OF_YEAR' column, year = 2015 it doesn't exist in the dataset
# 1 - 365; use MONTH (1-12), DAY (1-31), year (2015)
year = 2015
df_f['YEAR'] = year
df_f['day_of_year'] = pd.to_datetime(df_f[['YEAR', 'MONTH', 'DAY']]).dt.dayofyear
#remove the 'YEAR' column
df_f.drop('YEAR', axis=1, inplace=True)
sanity_check(df_f)

12 months
31 days
7 days of week
365 days of year


In [9]:
#head
df_f.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,SCHEDULED_DEPARTURE_TS,ACTUAL_DEPARTURE_TS,SCHEDULED_ARRIVAL_TS,ACTUAL_ARRIVAL_TS,day_of_year
0,1,1,4,AA,LAX,PBI,00:10,00:02,-8.0,12.0,...,737.0,4.0,07:50,07:41,-9.0,2015-01-01 00:10:00,2015-01-01 00:02:00,2015-01-01 07:50:00,2015-01-01 07:41:00,1
1,1,1,4,US,SFO,CLT,00:20,00:18,-2.0,16.0,...,800.0,11.0,08:06,08:11,5.0,2015-01-01 00:20:00,2015-01-01 00:18:00,2015-01-01 08:06:00,2015-01-01 08:11:00,1
2,1,1,4,AA,LAX,MIA,00:20,00:15,-5.0,15.0,...,748.0,8.0,08:05,07:56,-9.0,2015-01-01 00:20:00,2015-01-01 00:15:00,2015-01-01 08:05:00,2015-01-01 07:56:00,1
3,1,1,4,AS,SEA,ANC,00:25,00:24,-1.0,11.0,...,254.0,5.0,03:20,02:59,-21.0,2015-01-01 00:25:00,2015-01-01 00:24:00,2015-01-01 03:20:00,2015-01-01 02:59:00,1
4,1,1,4,DL,SFO,MSP,00:25,00:20,-5.0,18.0,...,604.0,6.0,06:02,06:10,8.0,2015-01-01 00:25:00,2015-01-01 00:20:00,2015-01-01 06:02:00,2015-01-01 06:10:00,1


In [12]:
#convert _valid_time_gmt to datetime, round to nearest hour
# i.e. 2015-01-02 04:51:00
df_w['valid_time_gmt'] = pd.to_datetime(df_w['valid_time_gmt'])
df_w['valid_time_gmt'] = df_w['valid_time_gmt'].dt.round('H')
#round to the nearest 2 hours
# df_w['valid_time_gmt'] = df_w['valid_time_gmt'].dt.round('H')
#convert all other times to datetime, round to nearest hour for flight data
#columns; 
df_w.head()

,day_of_year,time_of_day,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,dewPt,heat_index,rh,...,wdir,wdir_cardinal,wspd,precip_hrly,uv_desc,feels_like,uv_index,clds,wx_phrase_simple,airport
0,1.0,1.0,2015-01-02 01:00:00,N,41.0,26.0,2600.0,27.0,41.0,57.0,...,280.0,W,12.0,0.0,Low,34.0,0.0,OVC,Cloudy,ABE
1,1.0,2.0,2015-01-02 02:00:00,N,40.0,26.0,2600.0,27.0,40.0,60.0,...,280.0,W,12.0,0.0,Low,33.0,0.0,OVC,Cloudy,ABE
2,1.0,3.0,2015-01-02 03:00:00,N,39.0,26.0,2600.0,27.0,39.0,62.0,...,280.0,W,7.0,0.0,Low,34.0,0.0,OVC,Cloudy,ABE
3,1.0,4.0,2015-01-02 04:00:00,N,39.0,26.0,2600.0,27.0,39.0,62.0,...,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy,ABE
4,1.0,5.0,2015-01-02 05:00:00,N,39.0,26.0,2600.0,27.0,39.0,62.0,...,290.0,WNW,7.0,0.0,Low,34.0,0.0,OVC,Cloudy,ABE


In [14]:
#print unique day_of_year value count total unique
print(f'{len(df_w["day_of_year"].unique())} unique day_of_year')

365 unique day_of_year


In [14]:
#print columns
df_w.columns

Index(['day_of_year', 'time_of_day', 'valid_time_gmt', 'day_ind', 'temp',
       'wx_icon', 'icon_extd', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis',
       'wc', 'wdir', 'wdir_cardinal', 'wspd', 'precip_hrly', 'uv_desc',
       'feels_like', 'uv_index', 'clds', 'wx_phrase_simple', 'airport'],
      dtype='object')

In [15]:
# #get the minimum set of airports that are in both datasets
# lst_airports = list(set(df_f['ORIGIN_AIRPORT'].unique()).union(set(df_f['DESTINATION_AIRPORT'].unique())))
# lst_airports = list(set(lst_airports).intersection(set(df_w['airport'].unique())))
# #out of lst_airports, only keep the ones that have at least 365 days of data
# #in df_w, traverse through each airport, count the number of unique day_of_year, if it's 365, keep it
# print(len(lst_airports))
# #only keep the if airports are in the lst_airports
# print(len(df_f))
# df_f = df_f[df_f['ORIGIN_AIRPORT'].isin(lst_airports)]
# df_f = df_f[df_f['DESTINATION_AIRPORT'].isin(lst_airports)]
# print(len(df_f))
# df_w = df_w[df_w['airport'].isin(lst_airports)]

# #reset the index of df_f
# df_f.reset_index(drop=True, inplace=True)
# #reset the index of df_w
# df_w.reset_index(drop=True, inplace=True)
# # sanity_check(df_f)


In [18]:
sanity_check(df_f)
# sanity_check(df_w)

12 months
31 days
7 days of week
365 days of year


In [32]:
#print unique values of origin airport
print(len(df_f['ORIGIN_AIRPORT'].unique()))

931


In [29]:

#convert 'SCHEDULED_DEPARTURE_TS' and 'SCHEDULED_ARRIVAL_TS' to datetime, round to nearest hour
df_f['TIMESTAMP_HR'] = pd.to_datetime(df_f['SCHEDULED_DEPARTURE_TS'])
df_f['TIMESTAMP_HR'] = df_f['TIMESTAMP_HR'].dt.round('H')
#add 'airport' column to df_f
df_f['airport'] = df_f['ORIGIN_AIRPORT']
#add 'valid_time_gmt' column to df_f
df_w['TIMESTAMP_HR'] = df_w['valid_time_gmt']
#convert 'airport' column to category
df_f['airport'] = df_f['airport'].astype('category')
df_w['airport'] = df_w['airport'].astype('category')

#remove 'day_of_year' column from df_w
df_w.drop('day_of_year', axis=1, inplace=True)

#merge with a tolerance of 6 hours, keep left
df_m = pd.merge(
    left=df_f,
    right=df_w,
    how='left',
    left_on=['airport', 'TIMESTAMP_HR'],
    right_on=['airport', 'TIMESTAMP_HR'],
    suffixes=('', '_orig'),
    # on=None,
)


sanity_check(df_m)

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [17]:
#null
df_m.isnull().sum()
#total df length
# len(df_m)

MONTH                          0
DAY                            0
DAY_OF_WEEK                    0
AIRLINE                        0
ORIGIN_AIRPORT                 0
DESTINATION_AIRPORT            0
SCHEDULED_DEPARTURE            0
DEPARTURE_TIME                 0
DEPARTURE_DELAY                0
TAXI_OUT                       0
WHEELS_OFF                     0
SCHEDULED_TIME                 0
ELAPSED_TIME                   0
AIR_TIME                       0
DISTANCE                       0
WHEELS_ON                      0
TAXI_IN                        0
SCHEDULED_ARRIVAL              0
ARRIVAL_TIME                   0
ARRIVAL_DELAY                  0
SCHEDULED_DEPARTURE_TS         0
ACTUAL_DEPARTURE_TS            0
SCHEDULED_ARRIVAL_TS           0
ACTUAL_ARRIVAL_TS              0
day_of_year                    0
TIMESTAMP_HR                   0
airport                        0
time_of_day               477972
valid_time_gmt            477972
day_ind                   477972
temp      

In [18]:
#count unique months,day_of_year
print(len(df_m['MONTH'].unique()))
print(len(df_m['day_of_year'].unique()))

12
365


In [31]:
#print unique values of origin airport
print(len(df_m['ORIGIN_AIRPORT'].unique()))

931


In [30]:
#convert 'SCHEDULED_DEPARTURE_TS' and 'SCHEDULED_ARRIVAL_TS' to datetime, round to nearest hour
df_m['TIMESTAMP_HR'] = pd.to_datetime(df_m['SCHEDULED_ARRIVAL_TS'])
df_m['TIMESTAMP_HR'] = df_m['TIMESTAMP_HR'].dt.round('H')

#add 'airport' column to df_f
df_m['airport'] = df_m['DESTINATION_AIRPORT']

#convert 'airport' column to category
df_m['airport'] = df_m['airport'].astype('category')

#merge with a tolerance of 6 hours
df_m = pd.merge(
    left=df_m,
    right=df_w,
    how='left',
    on=['airport', 'TIMESTAMP_HR'],
    suffixes=('', '_dest'),
)

sanity_check(df_m)

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [20]:
#count unique months,day_of_year
print(len(df_m['MONTH'].unique()))
print(len(df_m['day_of_year'].unique()))

12
365


In [24]:
#print the only the columns that have null values, and their count vs the total count
# print(df_m.isnull().sum()[df_m.isnull().sum() > 0])
#fill wdir and wdir_dest with 0
df_m['wdir'].fillna(0, inplace=True)
df_m['wdir_dest'].fillna(0, inplace=True)
#remove clds, clds_dest, wdir_cardinal_dest rows with null values
if 'clds' in df_m.columns:
    df_m.drop('clds', axis=1, inplace=True)
if 'clds_dest' in df_m.columns:
    df_m.drop('clds_dest', axis=1, inplace=True)
if 'wdir_cardinal_dest' in df_m.columns:
    df_m.drop('wdir_cardinal_dest', axis=1, inplace=True)
#reset the index of df_m
df_m.reset_index(drop=True, inplace=True)
sanity_check(df_m)

12 months
31 days
7 days of week
365 days of year


In [25]:
#columns to drop
cols = ['valid_time_gmt', 'ACTUAL_ARRIVAL_TS', 'valid_time_gmt_dest', 'TIMESTAMP_HR', 'airport', 'ACTUAL_DEPARTURE_TS', 'SCHEDULED_DEPARTURE_TS', 'SCHEDULED_ARRIVAL_TS', 'wx_phrase', 'wx_phrase_dest']# 'ARRIVAL_TIME', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL', 'SCHEDULED_DEPARTURE']
#if thes columns are in the df, drop them
for col in cols:
    if col in df_m.columns:
        df_m.drop(col, axis=1, inplace=True)

sanity_check(df_m)

12 months
31 days
7 days of week
365 days of year


In [26]:
# #plot and save a frequency distribution of all the columns
# from matplotlib import pyplot as plt
# import os

# folder = 'plots'
# if not os.path.exists(folder):
#     os.makedirs(folder)
# for col in df_m.columns:
#     if col != 'diff':
#         df_m[col].hist()
#         plt.title(col)
#         plt.savefig(folder + '/' + col + '.png')
#         plt.clf()
#         plt.close()

In [27]:
# outliers to remove
# '''
# AIR_TIME #remove rows with AIR_TIME > 400
# ARRIVAL_DELAY #remove rows with ARRIVAL_DELAY > 500
# DEPARTURE_DELAY #remove rows with DEPARTURE_DELAY > 500
# dewPt #remove rows with dewPt > 100
# dewPt_dest #remove rows with dewPt_dest > 100
# DISTANCE #remove rows with DISTANCE > 3000
# SCHEDULED_TIME #remove rows with SCHEDULED_TIME > 500
# TAXI_IN #remove rows with TAXI_IN > 100
# TAXI_OUT #remove rows with TAXI_OUT > 100
# temp_dest #remove rows with temp_dest > 125
# temp #remove rows with temp > 125
# vis_dest #remove rows with vis_dest > 20
# vis #remove rows with vis > 20
# wc_dest #remove rows with wc_dest > 125
# wc #remove rows with wc > 125
# wspd_dest #remove rows with wspd_dest > 50
# wspd #remove rows with wspd > 50
# wx_phrase
# '''

#remove outliers
# df_m = df_m[df_m['AIR_TIME'] < 400]
# df_m = df_m[df_m['ARRIVAL_DELAY'] < 500]
# df_m = df_m[df_m['DEPARTURE_DELAY'] < 500]
# df_m = df_m[df_m['dewPt'] < 100]
# df_m = df_m[df_m['dewPt_dest'] < 100]
# df_m = df_m[df_m['DISTANCE'] < 3000]
# df_m = df_m[df_m['SCHEDULED_TIME'] < 500]
# df_m = df_m[df_m['TAXI_IN'] < 100]
# df_m = df_m[df_m['TAXI_OUT'] < 100]
# df_m = df_m[df_m['temp_dest'] < 125]
# df_m = df_m[df_m['temp'] < 125]
# df_m = df_m[df_m['vis_dest'] < 20]
# df_m = df_m[df_m['vis'] < 20]
# df_m = df_m[df_m['wc_dest'] < 125]
# df_m = df_m[df_m['wc'] < 125]
# df_m = df_m[df_m['wspd_dest'] < 50]
# df_m = df_m[df_m['wspd'] < 50]
# sanity_check(df_m)


In [28]:
#describe the data
df_m.describe()
m = dict_freq(df_m,col='MONTH')
for key in m:
    print(key, m[key])


1 441206
2 393381
3 470657
4 458170
5 466875
6 464186
7 485610
8 479521
9 445148
10 465155
11 441200
12 446166


In [37]:
#for each column collect total number of unique values and nulls
val = {}
for col in df_m.columns:
    val[col] = [len(df_m[col].unique()), df_m[col].isnull().sum()]

In [38]:
for key in val:
    print(f'{key}: {val[key]}')

MONTH: [12, 0]
DAY: [31, 0]
DAY_OF_WEEK: [7, 0]
AIRLINE: [14, 0]
ORIGIN_AIRPORT: [931, 0]
DESTINATION_AIRPORT: [929, 0]
SCHEDULED_DEPARTURE: [1282, 0]
DEPARTURE_TIME: [1437, 0]
DEPARTURE_DELAY: [1175, 0]
TAXI_OUT: [182, 0]
WHEELS_OFF: [1439, 0]
SCHEDULED_TIME: [519, 0]
ELAPSED_TIME: [712, 0]
AIR_TIME: [675, 0]
DISTANCE: [1347, 0]
WHEELS_ON: [1438, 0]
TAXI_IN: [179, 0]
SCHEDULED_ARRIVAL: [1283, 0]
ARRIVAL_TIME: [1433, 0]
ARRIVAL_DELAY: [1195, 0]
day_of_year: [365, 0]
time_of_day: [25, 477972]
day_ind: [3, 477972]
temp: [165, 477972]
wx_icon: [32, 477972]
icon_extd: [120, 477972]
dewPt: [126, 477972]
heat_index: [153, 477972]
rh: [101, 477972]
pressure: [893, 477972]
vis: [26, 477972]
wc: [186, 477972]
wdir: [42, 0]
wdir_cardinal: [20, 477972]
wspd: [57, 477972]
precip_hrly: [108, 477972]
uv_desc: [6, 477972]
feels_like: [187, 477972]
uv_index: [17, 477972]
wx_phrase_simple: [17, 477972]
time_of_day_dest: [25, 477953]
day_ind_dest: [3, 477953]
temp_dest: [165, 477953]
wx_icon_dest: [33, 

In [47]:
#print the unique values of time_of_day
print(df_m['time_of_day'].unique())

#remove rows with null values for column time_of_day
df_m = df_m[df_m['time_of_day'].notnull()]

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24.]


In [44]:
#print total unique values and nulls
for key in val:
    print(f'{key}: {val[key]}')
    

KeyboardInterrupt: 

In [48]:
# describe the data
df_m.describe()

,MONTH,DAY,DAY_OF_WEEK,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,...,dewPt_dest,heat_index_dest,rh_dest,pressure_dest,vis_dest,wc_dest,wspd_dest,precip_hrly_dest,feels_like_dest,uv_index_dest
count,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,...,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06,4.979303e+06
mean,6.231445e+00,1.572631e+01,3.924207e+00,8.549582e+00,1.607052e+01,1.331407e+03,1.392426e+02,1.344850e+02,1.109656e+02,7.978626e+02,...,4.749966e+01,6.316522e+01,6.297928e+01,2.914959e+01,9.233358e+00,6.098950e+01,8.513172e+00,2.194833e-03,6.170219e+01,2.035306e+00
std,3.376702e+00,8.743977e+00,1.991966e+00,3.497035e+01,8.920569e+00,4.734987e+02,7.281185e+01,7.176022e+01,6.972011e+01,5.775256e+02,...,1.897401e+01,2.031743e+01,2.180634e+01,1.348835e+00,2.119552e+00,2.197930e+01,5.445691e+00,1.989032e-02,2.282576e+01,2.248597e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,1.800000e+01,1.400000e+01,7.000000e+00,3.100000e+01,...,-3.800000e+01,-3.200000e+01,1.000000e+00,1.874000e+01,0.000000e+00,-6.600000e+01,0.000000e+00,0.000000e+00,-6.600000e+01,0.000000e+00
25%,3.000000e+00,8.000000e+00,2.000000e+00,-5.000000e+00,1.100000e+01,9.290000e+02,8.500000e+01,8.200000e+01,6.000000e+01,3.710000e+02,...,3.400000e+01,5.000000e+01,4.700000e+01,2.904000e+01,1.000000e+01,5.000000e+01,5.000000e+00,0.000000e+00,5.000000e+01,0.000000e+00
50%,6.000000e+00,1.600000e+01,4.000000e+00,-2.000000e+00,1.400000e+01,1.325000e+03,1.210000e+02,1.170000e+02,9.300000e+01,6.410000e+02,...,5.100000e+01,6.500000e+01,6.500000e+01,2.952000e+01,1.000000e+01,6.500000e+01,8.000000e+00,0.000000e+00,6.500000e+01,1.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,7.000000e+00,1.900000e+01,1.730000e+03,1.700000e+02,1.650000e+02,1.410000e+02,1.034000e+03,...,6.300000e+01,7.900000e+01,8.100000e+01,2.993000e+01,1.000000e+01,7.700000e+01,1.200000e+01,0.000000e+00,7.900000e+01,3.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,1.988000e+03,2.250000e+02,2.400000e+03,7.180000e+02,7.660000e+02,6.900000e+02,4.983000e+03,...,1.670000e+02,2.210000e+02,1.000000e+02,3.805000e+01,3.200000e+01,1.920000e+02,1.750000e+02,2.810000e+00,2.210000e+02,1.500000e+01


In [21]:
#save to csv
df_m.to_csv('../datasets/df_flight_weather_5.csv', index=False)

In [21]:
#for each column list the number of unique values
def unique_values(df):
    vals = {}
    for col in df.columns:
        vals[col] = df[col].nunique()
    return vals
val = unique_values(df_m)
for key in val:
    print(key, val[key])


MONTH 12
DAY 31
DAY_OF_WEEK 7
AIRLINE 14
ORIGIN_AIRPORT 931
DESTINATION_AIRPORT 929
SCHEDULED_DEPARTURE 1282
DEPARTURE_TIME 1437
DEPARTURE_DELAY 1175
TAXI_OUT 182
WHEELS_OFF 1439
SCHEDULED_TIME 519
ELAPSED_TIME 712
AIR_TIME 675
DISTANCE 1347
WHEELS_ON 1438
TAXI_IN 179
SCHEDULED_ARRIVAL 1283
ARRIVAL_TIME 1433
ARRIVAL_DELAY 1195
SCHEDULED_DEPARTURE_TS 378254
ACTUAL_DEPARTURE_TS 414810
SCHEDULED_ARRIVAL_TS 398693
ACTUAL_ARRIVAL_TS 408027
day_of_year 365
TIMESTAMP_HR 398693
airport 929
time_of_day 24
valid_time_gmt 3998
day_ind 2
temp 164
wx_icon 33
icon_extd 132
dewPt 128
heat_index 154
rh 101
pressure 903
vis 28
wc 186
wdir 48
wdir_cardinal 19
wspd 61
precip_hrly 215
uv_desc 5
feels_like 188
uv_index 16
wx_phrase_simple 16
time_of_day_dest 21
valid_time_gmt_dest 2988
day_ind_dest 2
temp_dest 128
wx_icon_dest 31
icon_extd_dest 89
dewPt_dest 105
heat_index_dest 128
rh_dest 100
pressure_dest 761
vis_dest 17
wc_dest 149
wdir_dest 38
wspd_dest 39
precip_hrly_dest 141
uv_desc_dest 5
feels_like

In [19]:
# describe
df_m.describe()

,MONTH,DAY,DAY_OF_WEEK,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,...,dewPt_dest,heat_index_dest,rh_dest,pressure_dest,vis_dest,wc_dest,wspd_dest,precip_hrly_dest,feels_like_dest,uv_index_dest
count,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,1.545744e+07,...,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000,693355.000000
mean,6.316509e+00,1.572264e+01,3.926721e+00,8.994709e+00,1.612569e+01,1.339223e+03,1.388128e+02,1.341422e+02,1.105486e+02,7.928493e+02,...,48.050541,61.440887,67.653971,29.242383,8.593420,59.167861,8.474255,0.008454,59.800632,1.882179
std,3.434754e+00,8.767700e+00,1.983569e+00,3.545867e+01,8.996024e+00,4.606698e+02,7.265447e+01,7.155088e+01,6.952557e+01,5.747356e+02,...,18.527905,19.992926,21.948977,1.204750,2.824185,21.850427,5.536374,0.054660,22.650897,2.168090
min,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,1.800000e+01,1.400000e+01,7.000000e+00,3.100000e+01,...,-26.000000,-13.000000,1.000000,22.090000,0.000000,-34.000000,0.000000,0.000000,-34.000000,0.000000
25%,3.000000e+00,8.000000e+00,2.000000e+00,-5.000000e+00,1.100000e+01,9.270000e+02,8.500000e+01,8.200000e+01,6.000000e+01,3.690000e+02,...,35.000000,48.000000,52.000000,29.090000,9.000000,45.000000,5.000000,0.000000,45.000000,0.000000
50%,6.000000e+00,1.600000e+01,4.000000e+00,-1.000000e+00,1.400000e+01,1.317000e+03,1.200000e+02,1.170000e+02,9.200000e+01,6.370000e+02,...,51.000000,63.000000,72.000000,29.540000,10.000000,63.000000,8.000000,0.000000,63.000000,1.000000
75%,9.000000e+00,2.300000e+01,6.000000e+00,7.000000e+00,1.900000e+01,1.717000e+03,1.700000e+02,1.650000e+02,1.400000e+02,1.025000e+03,...,64.000000,75.000000,86.000000,29.920000,10.000000,75.000000,12.000000,0.000000,75.000000,3.000000
max,1.200000e+01,3.100000e+01,7.000000e+00,1.988000e+03,2.250000e+02,2.400000e+03,7.180000e+02,7.660000e+02,6.900000e+02,4.983000e+03,...,83.000000,116.000000,100.000000,30.860000,13.000000,116.000000,52.000000,3.170000,116.000000,14.000000


In [20]:
#null values
df_m.isnull().sum()

MONTH                           0
DAY                             0
DAY_OF_WEEK                     0
AIRLINE                         0
ORIGIN_AIRPORT                  0
                           ...   
precip_hrly_dest         14764086
uv_desc_dest             14764086
feels_like_dest          14764086
uv_index_dest            14764086
wx_phrase_simple_dest    14764086
Length: 66, dtype: int64

In [315]:
#provide unique values for each column

val = {}
for col in df_m.columns:
    val[col] = df_m[col].nunique()
for key in val:
    print(key, val[key])

MONTH 12
DAY 31
DAY_OF_WEEK 7
AIRLINE 14
ORIGIN_AIRPORT 931
DESTINATION_AIRPORT 929
SCHEDULED_DEPARTURE 1282
DEPARTURE_TIME 1437
DEPARTURE_DELAY 1175
TAXI_OUT 182
WHEELS_OFF 1439
SCHEDULED_TIME 519
ELAPSED_TIME 712
AIR_TIME 675
DISTANCE 1347
WHEELS_ON 1438
TAXI_IN 179
SCHEDULED_ARRIVAL 1283
ARRIVAL_TIME 1433
ARRIVAL_DELAY 1195
day_of_year 365
day_ind 2
temp 166
wx_icon 33
icon_extd 129
dewPt 126
heat_index 156
rh 101
pressure 902
vis 26
wc 187
wdir 49
wdir_cardinal 19
wspd 63
precip_hrly 223
uv_desc 5
feels_like 189
uv_index 16
wx_phrase_simple 16
day_ind_dest 2
temp_dest 128
wx_icon_dest 30
icon_extd_dest 88
dewPt_dest 108
heat_index_dest 128
rh_dest 100
pressure_dest 753
vis_dest 16
wc_dest 149
wdir_dest 38
wspd_dest 38
precip_hrly_dest 150
uv_desc_dest 5
feels_like_dest 149
uv_index_dest 14
wx_phrase_simple_dest 16
